## Script para marcar as posicoes

In [1]:
import cv2 
import pickle
width, height = 100,45

try:
    with open("CarParkPos", "rb") as f:
        posList = pickle.load(f)
except:
    posList = []

def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x,y))
    if events == cv2.EVENT_RBUTTONDOWN:
        for i, pos in enumerate(posList):
            x1, y1 = pos
            if x1 < x < (x1 + width) and y1 < y < (y1+height):
                posList.pop(i)
    with open("CarParkPos", "wb") as f:
        pickle.dump(posList, f)

while True:
        key = cv2.waitKey(1) & 0xFF

        if key == 27:
            break
        img = cv2.imread("carParkImg.png")
        for pos in posList:
            cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255,0,255), 2)
        cv2.imshow("Image", img)
        cv2.setMouseCallback("Image", mouseClick)
        cv2.waitKey(1)
cv2.destroyAllWindows()



## Script para exibir o funcionamento da visão computacional das vagas

In [33]:
import cv2
import pickle
import cvzone
import numpy as np
video = cv2.VideoCapture("carPark.mp4")
velocidadeFrame = 3
color = (0,0,0)
with open("CarParkPos", "rb") as f:
    posList = pickle.load(f)

width, height = 107, 48

def checkParkingSpace(imgPro):
    spaceCounter = 0
    for pos in posList:
        x, y = pos
        imgCrop = imgPro[y: y + height, x : x +height]
        count = cv2.countNonZero(imgCrop)
        if count < 170:
            color = (0, 255, 0)
            thickness = 3
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)

        cvzone.putTextRect(img, str(count), (x,y+height-3), scale=1, thickness=2,offset=0,colorR=color)
    cvzone.putTextRect(img, f"VAGAS: {spaceCounter}/{len(posList)}", (100, 50), scale = 3, thickness=5, offset=20, colorR=(0,255,0))
    
def mouseClick(event, x, y, flags, param):
    global velocidadeFrame, color
    if event == cv2.EVENT_LBUTTONDOWN:
        velocidadeFrame += 100
        color = (0, 255, 0)
    elif event == cv2.EVENT_RBUTTONDOWN:
        velocidadeFrame = 3
        color = (0, 0, 255)

    

while True:
    try:
        key = cv2.waitKey(1) & 0xFF
        if(key == 27):
            break
        # DEIXA O VÍDEO EM LOOP
        if video.get(cv2.CAP_PROP_POS_FRAMES) == video.get(cv2.CAP_PROP_FRAME_COUNT):
            video.set(cv2.CAP_PROP_POS_FRAMES,0)
        success, img = video.read()
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray,(3,3),1)
        imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)

        imgMedian = cv2.medianBlur(imgThreshold,5)
        kernel = np.ones((3,3), np.uint8)
        imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)
        checkParkingSpace(imgDilate)
        cv2.imshow("Image", img)
        cv2.waitKey(velocidadeFrame)
        cv2.setMouseCallback("Image", mouseClick)
    except:
        print("Codigo encerrado")
        cv2.destroyAllWindows()
        break

cv2.destroyAllWindows()




Codigo encerrado
